# Attempt to fine-tune GPT3 to translate between languages
The first language will be our made up language (see notes on get_bible as to why we think GPT is cheating)
The thesis is that fine-tuning will cause GPT3 to learn the fake language as a new language by assigning the embeddings of the
new words essentially to the same dimensional space as the english.  Should be trivial to translate it back as it is a word 
for word translation.



In [2]:
from lib.config import get_config
import pandas as pd

bible = pd.read_csv('data/bible.bbe.csv')

# Assuming the gospels have a lot of overlap include the synoptic gospels plus John to train
# then predict Mark (which should be easy as Matthew and Luke may have copied from him)
train = bible[bible['book'].isin(['MAT','LUK','JHN'])]
validate = bible[bible['book'].isin(['MRK'])]

In [3]:
train

,0,book,chapter,verse,eng-web,eng-asv,eng-kjv2006,engBBE,engWycliffe,birrig
23213,MAT 1:1,MAT,1,1,"The book of the genealogy of Jesus Christ, the...","The book of the generation of Jesus Christ, th...","The book of the generation of Jesus Christ, th...","The book of the generations of Jesus Christ, t...","The book of the generacioun of Jhesu Crist, th...","Lxi fuuc us lxi girivelourz us Yizaz Kxvozl, l..."
23214,MAT 1:2,MAT,1,2,Abraham became the father of Isaac. Isaac beca...,Abraham begat Isaac; and Isaac begat Jacob; an...,Abraham begat Isaac; and Isaac begat Jacob; an...,The son of Abraham was Isaac; and the son of I...,Abraham bigat Isaac. Isaac bigat Jacob. Jacob ...,Lxi zur us Efvexen hez Ozeek; erw lxi zur us O...
23215,MAT 1:3,MAT,1,3,Judah became the father of Perez and Zerah by ...,and Judah begat Perez and Zerah of Tamar; and ...,And Judas begat Phares and Zara of Thamar; and...,And the sons of Judah were Perez and Zerah by ...,"Judas bigat Fares and Zaram, of Tamar. Fares b...",Erw lxi zurz us Yawex hivi Qivid erw Divex fj ...
23216,MAT 1:4,MAT,1,4,Ram became the father of Amminadab. Amminadab ...,and Ram begat Amminadab; and Amminadab begat N...,And Aram begat Aminadab; and Aminadab begat Na...,And the son of Ram was Amminadab; and the son ...,Esrom bigat Aram. Aram bigat Amynadab. Amynada...,Erw lxi zur us Ven hez Ennorewef; erw lxi zur ...
23217,MAT 1:5,MAT,1,5,Salmon became the father of Boaz by Rahab. Boa...,and Salmon begat Boaz of Rahab; and Boaz begat...,And Salmon begat Booz of Rachab; and Booz bega...,And the son of Salmon by Rahab was Boaz; and t...,"Salmon bigat Booz, of Raab. Booz bigat Obeth, ...",Erw lxi zur us Zepnur fj Vexef hez Fued; erw l...
...,...,...,...,...,...,...,...,...,...,...
26987,JHN 21:21,JHN,21,21,"Peter, seeing him, said to Jesus, “Lord, what ...","Peter therefore seeing him saith to Jesus, Lor...","Peter seeing him saith to Jesus, Lord, and wha...","Seeing him, Peter said to Jesus, What about th...","Therfor whanne Petre hadde seyn this, he seith...","Ziiorg xon, Qiliv zeow lu Yizaz, Hxel efual lx..."
26988,JHN 21:22,JHN,21,22,"Jesus said to him, “If I desire that he stay u...","Jesus saith unto him, If I will that he tarry ...","Jesus saith unto him, If I will that he tarry ...","Jesus said to him, If it is my desire for him ...","Jhesus seith to him, So I wole that he dwelle ...","Yizaz zeow lu xon, Os ol oz nj wizovi suv xon ..."
26989,JHN 21:23,JHN,21,23,This saying therefore went out among the broth...,This saying therefore went forth among the bre...,Then went this saying abroad among the brethre...,So this saying went about among the brothers t...,Therfor this word wente out among the brithere...,Zu lxoz zejorg hirl efual enurg lxi fvulxivz l...
26990,JHN 21:24,JHN,21,24,This is the disciple who testifies about these...,This is the disciple that beareth witness of t...,This is the disciple which testifieth of these...,This is the disciple who gives witness about t...,"This is thilke disciple, that berith witnessyn...",Lxoz oz lxi wozkoqpi hxu gomiz holrizz efual l...


In [20]:
df2 = pd.DataFrame({'prompt': train['eng-web'], 'completion': train['birrig']})
# remove rows in df2 that are not a string for
# prompt or completion
df2 = df2[df2['prompt'].apply(lambda x: isinstance(x, str))]
df2 = df2[df2['completion'].apply(lambda x: isinstance(x, str))]

df2['completion'] = df2['completion'].apply(lambda x: " " + x + '\n###')
df2['prompt'] = df2['prompt'].apply(lambda x: x + "\n\n###\n\n")
df2.to_json('data/train_exp2.jsonl', orient='records', lines=True)
df2


,prompt,completion
23213,"The book of the genealogy of Jesus Christ, the...","Lxi fuuc us lxi girivelourz us Yizaz Kxvozl, ..."
23214,Abraham became the father of Isaac. Isaac beca...,Lxi zur us Efvexen hez Ozeek; erw lxi zur us ...
23215,Judah became the father of Perez and Zerah by ...,Erw lxi zurz us Yawex hivi Qivid erw Divex fj...
23216,Ram became the father of Amminadab. Amminadab ...,Erw lxi zur us Ven hez Ennorewef; erw lxi zur...
23217,Salmon became the father of Boaz by Rahab. Boa...,Erw lxi zur us Zepnur fj Vexef hez Fued; erw ...
...,...,...
26987,"Peter, seeing him, said to Jesus, “Lord, what ...","Ziiorg xon, Qiliv zeow lu Yizaz, Hxel efual l..."
26988,"Jesus said to him, “If I desire that he stay u...","Yizaz zeow lu xon, Os ol oz nj wizovi suv xon..."
26989,This saying therefore went out among the broth...,Zu lxoz zejorg hirl efual enurg lxi fvulxivz ...
26990,This is the disciple who testifies about these...,Lxoz oz lxi wozkoqpi hxu gomiz holrizz efual ...


In [21]:

# set environment variable in ipython notebook
import os
os.environ["OPENAI_API_KEY"] = get_config('openai')['api_key']




In [22]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py tools fine_tunes.prepare_data -f data/train_exp2.jsonl -q

Analyzing...

- Your file contains 3100 prompt-completion pairs
- There are 1 duplicated prompt-completion sets. These are rows: [1395]
- All prompts end with suffix `\n\n###\n\n`
- All completions end with suffix `\n###`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 1 duplicate rows [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `data/train_exp2_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "data/train_exp2_prepared.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `\n\n###\n\n` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=["\n###"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 2.05 hours to train a `curie` model, and less for `ada` and `bab

In [23]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.create -t "data/train_exp2.jsonl"

Upload progress: 100%|███████████████████████| 870k/870k [00:00<00:00, 693Mit/s]
Uploaded file from data/train_exp2.jsonl: file-AtyEc40Ak0P6EOsHn7A0pua2
Created fine-tune: ft-2WbQFNqMyOAIgGot2iXlYR9C
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-29 13:51:59] Created fine-tune: ft-2WbQFNqMyOAIgGot2iXlYR9C

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-2WbQFNqMyOAIgGot2iXlYR9C



In [24]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.follow -i ft-2WbQFNqMyOAIgGot2iXlYR9C

[2023-03-29 13:51:59] Created fine-tune: ft-2WbQFNqMyOAIgGot2iXlYR9C
[2023-03-29 13:57:57] Fine-tune costs $3.50
[2023-03-29 13:57:57] Fine-tune enqueued. Queue number: 0
[2023-03-29 13:57:58] Fine-tune started
[2023-03-29 14:04:47] Completed epoch 1/4
[2023-03-29 14:10:33] Completed epoch 2/4
[2023-03-29 14:16:20] Completed epoch 3/4
[2023-03-29 14:22:06] Completed epoch 4/4
[2023-03-29 14:22:30] Uploaded model: curie:ft-personal-2023-03-29-21-22-30
[2023-03-29 14:22:31] Uploaded result file: file-CDG8qTXI2YmF60PYgFF7GsoN
[2023-03-29 14:22:31] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2023-03-29-21-22-30 -p <YOUR_PROMPT>


In [29]:
import openai
from lib.cipher import substitution_cipher
from nltk.translate.bleu_score import sentence_bleu

openai.api_key = os.getenv("OPENAI_API_KEY")

def translate(text):
    response = openai.Completion.create(
        model="curie:ft-personal-2023-03-29-21-22-30",
        prompt=text + "\n\n###\n\n",
        temperature=0.5,  # 0.1 is very little randomness/creativity, 2 is very
        max_tokens=100,
        n=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["\n###"]
    )
    result = []
    for choice in response.get('choices',[{}]):
        decoded = substitution_cipher(choice.get('text').strip(), encode=False).strip()
        result.append({
            'translation': choice.get('text').strip(),
            'decoded_translation': decoded,
            'bleu_score': sentence_bleu([text.split()], decoded.split())
        })
    
    return result

In [26]:
translate("For God so loved me that he gave me new life in his kingdom")

[{'translation': 'Suv Guw zu xew pumi suv ni, zu lxel xi gemi ni rih posi or xoz corgwun',
  'decoded_translation': 'For God so had love for me, so that he gave me new life in his kingdom',
  'bleu_score': 0.5635190098079903},
 {'translation': 'Suv Guw, wizovorg ni, xi gemi ni rih posi or xoz corgwun',
  'decoded_translation': 'For God, desiring me, he gave me new life in his kingdom',
  'bleu_score': 0.5345999665313225},
 {'translation': 'Suv Guw xew pumi or nj pogxl lxel xi gemi ni rih posi or xoz corgwun',
  'decoded_translation': 'For God had love in my light that he gave me new life in his kingdom',
  'bleu_score': 0.5554570250728591}]

In [27]:
translate("For God so loved the world that he gave his only Son, so that everyone who believes in him may not die but have eternal life.")

[{'translation': 'Suv Guw xew pumi suv lxi huvpw lxel xi gemi xoz urpj zur, zu lxel imivjuri hxu xez seolx or xon nej rul kuni lu wielx fal xemi ilivrep posi.',
  'decoded_translation': 'For God had love for the world that he gave his only son, so that everyone who has faith in him may not come to death but have eternal life.',
  'bleu_score': 0.44534567460430624},
 {'translation': 'Suv Guw zu xew pumi suv lxi huvpw lxel xi gemi xoz urpj Zur, zu lxel imivjuri hxu xez seolx or xon nej rul kuni lu wielx fal xemi ilivrep posi.',
  'decoded_translation': 'For God so had love for the world that he gave his only Son, so that everyone who has faith in him may not come to death but have eternal life.',
  'bleu_score': 0.5558807065436926},
 {'translation': 'Suv Guw zu xew pumi suv lxi huvpw zu lxel xi gemi xoz urpj Zur, zu lxel imivjuri hxu xez seolx or xon nej rul kuni lu wielx fal xemi ilivrep posi.',
  'decoded_translation': 'For God so had love for the world so that he gave his only Son, so

In [30]:
translate("he said let there be light and there was light")



[{'translation': 'Erw xi zeow pil lxivi fi pogxl erw lxivi hez pogxl',
  'decoded_translation': 'And he said let there be light and there was light',
  'bleu_score': 0.8931539818068694}]

In [ ]:
# Try some unknown words
translate("Bongo bongo I love you, gone to Venus with a hole in my shoe")

In [31]:
translate ("Ship Pit! Pirate ahoy-lay!")

[{'translation': 'Erw nir leci xon suv heliv,',
  'decoded_translation': 'And men take him for water,',
  'bleu_score': 0}]

In [32]:
translate("Sheep went baa")

/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

[{'translation': 'Erw zxiiq hirl wuhr efual.',
  'decoded_translation': 'And sheep went down about.',
  'bleu_score': 1.2183324802375697e-231}]

In [33]:
translate("Boat a brother on a mountain")

[{'translation': 'Erw lxi fvulxiv ur lxi nuarleor,',
  'decoded_translation': 'And the brother on the mountain,',
  'bleu_score': 7.57965434483665e-155}]

In [34]:
translate("Be kind and play with rocks!")

[{'translation': 'Xemi pumi ur suvn erw leci holx zluriwz.',
  'decoded_translation': 'Have love on form and take with stoneds.',
  'bleu_score': 1.2882297539194154e-231}]

In [35]:
translate("SOS! Ship overboard!! Lost ninty percent of people!")

[{'translation': 'Ziiw ziir, ziiw ziir, ziiw ziir, ual uvwivz!',
  'decoded_translation': 'Seed seen, seed seen, seed seen, out orders!',
  'bleu_score': 0}]